In [27]:
# -*- coding: utf-8 -*-
import sys

import pandas

import time
import datetime
import csv



path = "D:/User/Desktop/Insta/Hashtags.xlsx"

df = pandas.read_excel(path,'Hashtags')
#df_vars = pandas.read_excel(path,'Vars')

df["Hashtag"] = df["Hashtag"].str.lower()

In [28]:
df = df.drop_duplicates(subset="Hashtag", keep='first')

## Generate hashtags based on type

In [29]:
df.to_excel(path,index=False)

,Name,Description,Hashtags,Post Date
0,NaN,NaN,NaN,NaT
1,NaN,NaN,NaN,NaT
2,NaN,NaN,NaN,NaT
3,NaN,NaN,NaN,NaT
4,NaN,NaN,NaN,NaT
...,...,...,...,...
267,NaN,NaN,NaN,NaT
268,NaN,NaN,NaN,NaT
269,NaN,NaN,NaN,NaT
270,NaN,NaN,NaN,NaT
